**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**

---


# Introduction

You are getting to the point where you can own an analysis from beginning to end. So you'll do more data exploration in this exercise than you've done before.  Before you get started, run the following set-up code as usual. 

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex5 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


You'll work with a dataset about taxi trips in the city of Chicago. Run the cell below to fetch the `chicago_taxi_trips` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

You are curious how much slower traffic moves when traffic volume is high. This involves a few steps.

### 1) Find the data
Before you can access the data, you need to find the table name with the data.

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [3]:
# Your code here to find the table name
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)

taxi_trips


In [4]:
# Write the table name as a string below
table_name = 'taxi_trips'

# Check your answer
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [5]:
#q_1.solution()

### 2) Peek at the data

Use the next code cell to peek at the top few rows of the data. Inspect the data and see if any issues with data quality are immediately obvious. 

In [6]:
# Your code here
# # Construct a reference to the "taxi_trips" table
# table_ref = dataset_ref.table("taxi_trips")

# # API request - fetch the table
# table = client.get_table(table_ref)

# Preview the first five lines of the "taxi_trips" table
client.list_rows(table, max_results=5).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,81d6112c1bbd0187eb51167bda64e3856e41efc9,49692f79df4c1c5856e2568d485fd41a63acc6e2b16b5c...,2018-03-06 17:00:00+00:00,2018-03-06 17:15:00+00:00,1182,4.5,None,None,14,None,15.75,0.0,None,0.0,15.75,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
1,74af979c83c7681decefae3248c4be53eb83dd75,90a7cf3946c408e70e8d64b08f2bc6819ae5de6159ecef...,2018-03-06 09:00:00+00:00,2018-03-06 09:15:00+00:00,1009,4.6,None,None,14,None,14.75,0.0,None,0.0,14.75,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
2,6975ab2efb215e30f55c6b43e96dea23b19ccd98,a0a5ca80f20ed5f103270c7c1d197539033a94b60c6bbd...,2018-03-12 22:00:00+00:00,2018-03-12 22:30:00+00:00,1569,20.3,None,None,14,None,49.75,0.0,None,0.0,49.75,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
3,628ce3fb7694f62f32a1bf22def72e369f216317,b597697c5b962a3f36ed67d274ec82ed1b72232c537b5a...,2018-03-12 05:00:00+00:00,2018-03-12 05:15:00+00:00,840,7.9,None,None,14,None,21.75,0.0,None,0.0,21.75,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
4,bf1828ce376a873a37b390a2d348a102da260bf9,d9b418798eff38cad71ed49fc9fa217dbe1aa91d4b6fd7...,2018-03-14 14:30:00+00:00,2018-03-14 15:00:00+00:00,1054,4.1,None,None,14,None,14.25,0.0,None,0.0,14.25,Credit Card,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None


After deciding whether you see any important issues, run the code cell below.

In [7]:
#q_2.solution()

### 3) Determine when this data is from

If the data is sufficiently old, we might be careful before assuming the data is still relevant to traffic patterns today. Write a query that counts the number of trips in each year.  

Your results should have two columns:
- `year` - the year of the trips
- `num_trips` - the number of trips in that year

Hints:
- When using **GROUP BY** and **ORDER BY**, you should refer to the columns by the alias `year` that you set at the top of the **SELECT** query.
- The SQL code to **SELECT** the year from `trip_start_timestamp` is <code>SELECT **EXTRACT(YEAR FROM trip_start_timestamp)**</code>
- The **FROM** field can be a little tricky until you are used to it.  The format is:
    1. A backick (the symbol \`).
    2. The project name. In this case it is `bigquery-public-data`.
    3. A period.
    4. The dataset name. In this case, it is `chicago_taxi_trips`.
    5. A period.
    6. The table name. You used this as your answer in **1) Find the data**.
    7. A backtick (the symbol \`).

In [8]:
# Your code goes here
rides_per_year_query = """
                        SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year, COUNT(1) AS num_trips
                        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                        GROUP BY year
                        ORDER BY year
                       """

# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_year_query_job = client.query(rides_per_year_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
rides_per_year_result = rides_per_year_query_job.to_dataframe() # Your code goes here

# View results
print(rides_per_year_result)

# Check your answer
q_3.check()

   year  num_trips
0  2013   27217716
1  2014   37395436
2  2015   32385875
3  2016   31759339
4  2017   24988003
5  2018   20732088
6  2019    7032967


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [9]:
#q_3.hint()
#q_3.solution()

### 4) Dive slightly deeper

It's odd that 2017 had so few rides. You should wonder whether it was systematic under-reporting throughout the year, or whether some months are missing.  Copy the query you used above in `rides_per_year_query` into the cell below for `rides_per_month_query`.  Then modify it in two ways:
1. Use a **WHERE** clause to limit the query to data from 2017.
2. Modify the query to extract the month rather than the year.

In [10]:
# Your code goes here
rides_per_month_query = """
                        SELECT EXTRACT(MONTH FROM trip_start_timestamp) AS month, COUNT(1) AS num_trips
                        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                        WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2017
                        GROUP BY month
                        ORDER BY month
                       """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_month_query_job = client.query(rides_per_month_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
rides_per_month_result = rides_per_month_query_job.to_dataframe() # Your code goes here

# View results
print(rides_per_month_result)

# Check your answer
q_4.check()

    month  num_trips
0       1    1972071
1       2    1909802
2       3    2362105
3       4    2194702
4       5    2323386
5       6    2324472
6       7    2054299
7       8    2079861
8       9    1950631
9      10    2141197
10     11    1907997
11     12    1767480


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [11]:
#q_4.hint()
#q_4.solution()

### 5) Write the query

It's time to step up the sophistication of your queries.  Write a query that shows, for each hour of the day in the dataset, the corresponding number of trips and average speed.

Your results should have three columns:
- `hour_of_day` - sort by this column, which holds the result of extracting the hour from `trip_start_timestamp`.
- `num_trips` - the count of the total number of trips in each hour of the day (e.g. how many trips were started between 6AM and 7AM, independent of which day it occurred on).
- `avg_mph` - the average speed, measured in miles per hour, for trips that started in that hour of the day.  Average speed in miles per hour is calculated as `3600 * SUM(trip_miles) / SUM(trip_seconds)`. (The value 3600 is used to convert from seconds to hours.)

For 2017, we're missing August and everything after. So restrict your query to data meeting the following criteria:
- a `trip_start_timestamp` between **2017-01-01** and **2017-07-01**
- `trip_seconds` > 0 and `trip_miles` > 0

You will use a common table expression (CTE) to select just the relevant rides.  Because this dataset is very big, this CTE should select only the columns you'll need to create the final output (though you won't actually create those in the CTE -- instead you'll create those in the later **SELECT** statement below the CTE).

This is a much harder query than anything you've written so far.  Good luck!

In [12]:
# Your code goes here
speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day, 
                          trip_miles, 
                          trip_seconds
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_start_timestamp > '2017-01-01' AND 
                         trip_start_timestamp < '2017-07-01' AND 
                         trip_seconds > 0 AND 
                         trip_miles > 0
               )
               SELECT hour_of_day, 
                      COUNT(1) AS num_trips, 
                      3600 * SUM(trip_miles) / SUM(trip_seconds) AS avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
speeds_query_job = client.query(speeds_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
speeds_result = speeds_query_job.to_dataframe()
# View results
print(speeds_result)

# Check your answer
q_5.check()

    hour_of_day  num_trips    avg_mph
0             0     319339  20.230524
1             1     266529  18.937621
2             2     210147  18.777070
3             3     159668  20.158048
4             4     122183  26.736014
5             5     119312  30.769172
6             6     182738  24.588313
7             7     358406  17.735967
8             8     541775  15.079892
9             9     565548  16.543882
10           10     525120  18.539614
11           11     594603  18.928379
12           12     622324  17.838745
13           13     630181  17.671089
14           14     622465  16.974239
15           15     640430  15.688418
16           16     701435  14.283888
17           17     756627  12.462955
18           18     768251  13.646810
19           19     701064  16.642882
20           20     598614  19.536777
21           21     552726  20.433874
22           22     501095  19.531374
23           23     399587  19.877046


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the appropriate line below.

In [13]:
#q_5.solution()

That's a hard query. If you made good progress towards the solution, congratulations!

### 6) Ponder the results
Something is wrong with either the raw data or our last query. What fact about the raw data doesn't seem right?

If you can identify the problem, how would you look at the raw data to verify that the problem is in the raw data and not just in your results? Check your answer below.

In [14]:
q_6.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
The results show rides with hours 1-12. But there should be results in the afternoon (hours 13-24).

Perhaps the raw data has lost the distinction between AM and PM values.

You can review 200 rows of the raw data with the commands: 
```python
# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "taxi_trips" table
client.list_rows(table, max_results=200).to_dataframe()
```

You'll see that the timestamps are all in the AM hours (hours are less than or equal to 12.) 

At first you might worry that the data is coming back sorted by time, but the variety of dates suggests that's not the case. 
Part of data science is tracking down exactly this type of problem. If you were in an organization working on this, you could show the evidence you've just collected (e.g. the breakdown of trips by hour) to someone responsible for collecting the data, and help them debug the data collection and storage process.


In [15]:
# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "taxi_trips" table
client.list_rows(table, max_results=200).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,81d6112c1bbd0187eb51167bda64e3856e41efc9,49692f79df4c1c5856e2568d485fd41a63acc6e2b16b5c...,2018-03-06 17:00:00+00:00,2018-03-06 17:15:00+00:00,1182,4.5,None,None,14,NaN,15.75,0.0,None,0.0,15.75,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),NaN,NaN,None
1,74af979c83c7681decefae3248c4be53eb83dd75,90a7cf3946c408e70e8d64b08f2bc6819ae5de6159ecef...,2018-03-06 09:00:00+00:00,2018-03-06 09:15:00+00:00,1009,4.6,None,None,14,NaN,14.75,0.0,None,0.0,14.75,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),NaN,NaN,None
2,6975ab2efb215e30f55c6b43e96dea23b19ccd98,a0a5ca80f20ed5f103270c7c1d197539033a94b60c6bbd...,2018-03-12 22:00:00+00:00,2018-03-12 22:30:00+00:00,1569,20.3,None,None,14,NaN,49.75,0.0,None,0.0,49.75,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),NaN,NaN,None
3,628ce3fb7694f62f32a1bf22def72e369f216317,b597697c5b962a3f36ed67d274ec82ed1b72232c537b5a...,2018-03-12 05:00:00+00:00,2018-03-12 05:15:00+00:00,840,7.9,None,None,14,NaN,21.75,0.0,None,0.0,21.75,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),NaN,NaN,None
4,bf1828ce376a873a37b390a2d348a102da260bf9,d9b418798eff38cad71ed49fc9fa217dbe1aa91d4b6fd7...,2018-03-14 14:30:00+00:00,2018-03-14 15:00:00+00:00,1054,4.1,None,None,14,NaN,14.25,0.0,None,0.0,14.25,Credit Card,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),NaN,NaN,None
5,6f5df6e4de83319c55a7f8fac895e84036fe7324,e1923f4173918030933c6977c31d14ce3842c601be74aa...,2018-03-17 01:30:00+00:00,2018-03-17 01:45:00+00:00,409,2.4,None,None,14,NaN,9.00,0.0,None,0.0,9.00,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),NaN,NaN,None
6,1471ce23b7a7a28cef9a392122b15e17094bfec3,a9f0f4f8c94582431a8bfd84c5125c509354fad6e87adb...,2018-03-10 22:00:00+00:00,2018-03-10 22:30:00+00:00,1231,9.4,None,None,14,NaN,26.00,0.0,None,0.0,26.00,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),NaN,NaN,None
7,27d49e84c4323eb99d422b9bc09d571833f2974d,b597697c5b962a3f36ed67d274ec82ed1b72232c537b5a...,2018-03-17 22:45:00+00:00,2018-03-17 23:00:00+00:00,1154,8.5,None,None,14,NaN,23.75,0.0,None,0.0,23.75,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),NaN,NaN,None
8,ac1bf0da9c359eecf1a1d06f10ea29ef69aa36dc,c0b49a3ce22900ce689ca45bc7d0b24d8b2d9604d4f790...,2018-03-10 14:00:00+00:00,2018-03-10 14:30:00+00:00,1318,7.0,None,None,14,NaN,21.00,0.0,None,0.0,21.00,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),NaN,NaN,None
9,2d7bc5024bc738524416320dac261047d9cdbd3c,b597697c5b962a3f36ed67d274ec82ed1b72232c537b5a...,2018-03-16 13:00:00+00:00,2018-03-16 13:15:00+00:00,1214,5.7,None,None,14,NaN,17.50,0.0,None,0.0,17.50,Cash,Flash Cab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),NaN,NaN,None


# Keep going

You can write very complex queries now with a single data source. But nothing expands the horizons of SQL as much as the ability to combine or **JOIN** tables.

**[Click here](https://www.kaggle.com/dansbecker/joining-data)** to start the last lesson in the SQL micro-course.

---
**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*